# Notebook 4: Adding Image Post-Processing

Example: Acquire Process Decide
Acquisitions, Processes, and decisions cann be prebuilt from libraries and stiched 

The AcquisitonPLugin is a set of variables and callback functions that can be used to acquire a dataset.

The PostProcesingPipeline takes a dataset and returns ProcessedData. 

The Deicision takes ProcessedData and returns an acquisition.

This 3 step process enables advanced automation of image acquistion. 

In [1]:
import sys,os
import warnings
import copy
sys.path.append('source')
from source.apd import *
from source.postprocessors import PostProcessPipeline as PostProcessPipeline
from source.postprocessors import PostProcessNode

from source.hooks import HookSet, Hook
from source.hooks import ImageHook

/Users/munsky/Dropbox/My Mac (ETSs-MacBook-Pro.local)/Documents/GitHub/MicroscopyControlAndProcessing/venv/lib/python3.11/site-packages/docstring_inheritance/docstring_inheritors/bases/inheritor.py:329: DocstringInheritanceWarning: File /Users/munsky/Dropbox/My Mac (ETSs-MacBook-Pro.local)/Documents/GitHub/MicroscopyControlAndProcessing/venv/lib/python3.11/site-packages/pycromanager/acquisitions.py:698: in XYTiledAcquisition.__init__: section Parameters: the docstring for the argument 'tile_overlap' is missing.
  self._warn(
/Users/munsky/Dropbox/My Mac (ETSs-MacBook-Pro.local)/Documents/GitHub/MicroscopyControlAndProcessing/venv/lib/python3.11/site-packages/docstring_inheritance/docstring_inheritors/bases/inheritor.py:329: DocstringInheritanceWarning: File /Users/munsky/Dropbox/My Mac (ETSs-MacBook-Pro.local)/Documents/GitHub/MicroscopyControlAndProcessing/venv/lib/python3.11/site-packages/pycromanager/acquisitions.py:698: in XYTiledAcquisition.__init__: section Parameters: the docstr

In [15]:
# Function to create a grid of initial imaging positions
def makeGrid(xRange,yRange):
    grid = []
    for x in xRange:
        if x % 2 == 0:
            yr = yRange
        else:
            yr = yRange[::-1]
        for y in yr:
            grid.append([x,y])
    return grid

In [16]:
# Set up the INITIAL aquisition object.  This will define which positions to acquire at.
lib = AcquisitionPluginLibrary()
positions = makeGrid(np.linspace(0,3,5),np.linspace(0,3,5))
acquisition = lib.xySequence(positions)
acquisition.settings.show_display=False

# Define an image processing routine, which returns the processed image and metadata.
# This code is meant to run immediatedly after the image is acquired, so it should be fast.
def imageProcessFunction(self,image,metadata,**kwargs):
    image = image[200:300,200:300]
    # add random noise to the image
    image = image + np.random.normal(0,100,image.shape).astype(np.uint16)
    metadata['Height'], metadata['Width'] = image.shape
    return image, metadata

# Attaching the image processing function to the acquisition object
hooks=HookSet()
hooks.hookImageProcess=ImageHook(function=imageProcessFunction)
acquisition.hooks=hooks

# Define an image post-processing routine, which returns a dictionary of data chunks and their processed data.
# This code is meant to run after all images have been acquired, so it can be slow.
def computeMean(self,chunks,acq,*args,**kwargs):
    output={'chunkMean':[chunk.mean() for chunk in chunks]}
    return (chunks,output)

# Putting the post process function into the Acquisition/Process pipeline
process=PostProcessPipeline()
process.addNode(PostProcessNode(function=computeMean))

# Define a function to generate new acquisition object based on the processed data.
def acquisitionDownSelect(processedData,acquisition):
    # Parse the processed data into a readable list.
    dataMetric = parseProcessedData(processedData[0],dataFields=['chunkMean'])[:,2]
    # find the n brightest images and return the corresponding acquisition object for just those positions
    n = int(len(dataMetric)/2)
    brightestIndices = np.argsort(dataMetric)[-n:].tolist()
    newAcquisition = copy.deepcopy(acquisition)
    newAcquisition.events.xy_positions = [newAcquisition.events.xy_positions[i] for i in brightestIndices]
    return newAcquisition   

In [21]:
#Set up and run the Initial Acquisition and Process Routine
apdSystem = APDSystem(configFileName='myConfig.cfg', rootDataFolder='') #local
apdSystem.linkAPD(acquisition=acquisition,process=process,decision=[])
processedData = apdSystem.run()

# Parse the processed data into a readable list.
#parsedList = parseProcessedData(processedData[0],dataFields=['chunkMean'])
#print(f'Initial parsed list is {parsedList}')

In [22]:
# Generate the new acquisition object
newAcquisition = acquisitionDownSelect(processedData,acquisition)

# Set up and run the Subsequent Acquisition and Process Routine
apdSystem = APDSystem(configFileName='myConfig.cfg', rootDataFolder='') #local
apdSystem.linkAPD(acquisition=newAcquisition,process=process,decision=[])
processedData = apdSystem.run()
#parsedList = parseProcessedData(processedData[0],dataFields=['chunkMean'])
#print(f'Final parsed list is {parsedList}')

In [25]:
def runSequence(Inital,Subsequent):
    
    # Set up the INITIAL aquisition object.  This will define which positions to acquire at.
    lib = AcquisitionPluginLibrary()
    positions = Initial['acquisition']['positions']
    acquisition = lib.xySequence(positions)
    acquisition.settings.show_display=False

    # Attaching the image processing function to the acquisition object
    hooks=HookSet()
    hooks.hookImageProcess=ImageHook(function=Initial['acquisition']['imageProcessFunction'])
    acquisition.hooks=hooks

    # Putting the post process function into the Acquisition/Process pipeline
    process=PostProcessPipeline()
    process.addNode(PostProcessNode(function=Initial['process']))

    #Set up and run the Initial Acquisition and Process Routine
    apdSystem = APDSystem(configFileName='myConfig.cfg', rootDataFolder='') #local
    apdSystem.linkAPD(acquisition=acquisition,process=process,decision=[])
    processedData = apdSystem.run()

    parsedList = parseProcessedData(processedData[0],dataFields=['chunkMean'])
    print(f'Initial parsed list is {parsedList}')
    
    for subseq in Subsequent:
        # Generate the new acquisition object
        newAcquisition = subseq['acquisitionDownSelect'](processedData,acquisition)

        # Attaching the image processing function to the acquisition object
        hooks=HookSet()
        hooks.hookImageProcess=ImageHook(function=subseq['imageProcessFunction'])
        acquisition.hooks=hooks
        
        # Set up and run the Subsequent Acquisition and Process Routine
        apdSystem = APDSystem(configFileName='myConfig.cfg', rootDataFolder='')
        apdSystem.linkAPD(acquisition=newAcquisition,process=subseq['process'] ,decision=[])
        processedData = apdSystem.run()

        parsedList = parseProcessedData(processedData[0],dataFields=['chunkMean'])
        print(f'\nFinal parsed list is {parsedList}')
 
    return processedData

positions = makeGrid(np.linspace(0,3,3),np.linspace(0,3,3))
Initial = {'acquisition':{'positions':positions,'imageProcessFunction':imageProcessFunction},
           'process':computeMean}
Subsequent = [{'acquisitionDownSelect':acquisitionDownSelect,'imageProcessFunction':imageProcessFunction,
            'process':process}]

runSequence(Initial,Subsequent)

Initial parsed list is [[0.00000000e+00 0.00000000e+00 5.20929853e+04]
 [1.00000000e+00 0.00000000e+00 4.89106715e+04]
 [2.00000000e+00 0.00000000e+00 3.62732405e+04]
 [3.00000000e+00 0.00000000e+00 2.15789403e+04]
 [4.00000000e+00 0.00000000e+00 1.34401241e+04]
 [5.00000000e+00 0.00000000e+00 1.66221491e+04]
 [6.00000000e+00 0.00000000e+00 2.92604768e+04]
 [7.00000000e+00 0.00000000e+00 4.39523672e+04]
 [8.00000000e+00 0.00000000e+00 5.20916806e+04]]

Final parsed list is [[0.00000000e+00 0.00000000e+00 4.89115149e+04]
 [1.00000000e+00 0.00000000e+00 3.62729734e+04]
 [2.00000000e+00 0.00000000e+00 2.15822169e+04]
 [3.00000000e+00 0.00000000e+00 1.34381503e+04]]


[[<source.postprocessors.OutputMapper at 0x14cee1090>]]

## Example 4.1: Using the APD System to run an AcquisitionPlugin-PostProcessPipeline-Decision (APD)

## Example 4.X: Writing your own PostProcess to print Hello World

This section describes how to make your own PostProcessNode using a function callback.

The first section describes the callback schema for process node functions, and the secon section shows how to run the callbacks.


In [ ]:
import copy

print((acquisition.events.xy_positions))
# copy acquisition to newAcquisition

newAcquisition=copy.deepcopy(acquisition)
newAcquisition.events.xy_positions = newAcquisition.events.xy_positions[0:2]
print(newAcquisition.events.xy_positions)
print(acquisition.events.xy_positions)



In [ ]:
from source.apd import *
acqLibrary=AcquisitionPluginLibrary()
acquisition=acqLibrary.xySequence([[0,0]])

process=PostProcessPipeline()
process.add('mean')

decision=DecisionRepeatAcquisition()

apdSystem = APDSystem(configFileName='myConfig.cfg', rootDataFolder='') 
apdSystem.linkAPD(acquisition=acquisition,process=process,decision=DecisionRepeatAcquisition())
apdSystem.run()


In [ ]:
print(dataset)

In [ ]:
newApdSystem = APDSystem(configFileName='myConfig.cfg', rootDataFolder='') #local
newApdSystem.linkAPD(acquisition=newAcquisition,process=process,decision=decision)
newApdSystem.run()

In [ ]:
def cropImage(self,chunks,acq,*args,**kwargs):
    output={}
    for i in range(len(chunks)):
        chunks[i]=chunks[i][10:,:-10] 
    return (chunks,output)

def averageImageValue(self,chunks,acq,*args,**kwargs):
    output={}
    for i in range(len(chunks)):
        print(chunks[i].shape)
        output['averageImageValue']=np.mean(chunks[i])
    return (chunks,output)


# Putting the functions into the pipeline and running it

processors=PostProcessPipeline()
processors.addNode(PostProcessNode(function=cropImage))
processors.addNode(PostProcessNode(function=averageImageValue))

# Running the function. <data> and <acq> are just arbitrary things needed to get the callback to work in this particular scenario.

data=NDTiffDataset('data/core/xyLooseGrid_1') # Load Sample dataset
acq=AcquisitionPlugin() #make an empty acquisition to pass into for testing
processors.process(data,acq)

## Example X.X: Using Cell Detection on Two Color Images

In [ ]:
def detectCells(self,chunks,acq,*args,**kwargs):
    output={}
    detector=CellDetectorCellMask()
    for i in range(len(chunks)):
        print(chunks[i].shape)
        output['averageImageValue']=detector.process(chunks[i])
    return (chunks,output)

# Putting the functions into the pipeline and running it

processors=PostProcessPipeline()
processors.addNode(PostProcessNode(function=detectCells,squish_axes='channel')) # squish_axes combines images long the channel dimension

# Running the function. <data> and <acq> are just arbitrary things needed to get the callback to work in this particular scenario.

data=NDTiffDataset('data/core/xyLooseGrid_5') # Load Sample dataset thats is 2 channel
acq=AcquisitionPlugin() #make an empty acquisition to pass into for testing
processedData=processors.process(data,acq)

ADD DISTRIBUTED COMPUTING!!!!!

TRANSFRER runners to BRIANS 